In [ ]:
'CF002327' - удалил 2 фотки
'CF002370' - разобрал по папкам A B - переименовал заказы
'CF002372' - разобрал по папкам A B C D - переименовал заказы
'CF002489', - разобрал по папкам A B C D - переименовал заказы
'CF002544', - разобрал по папкам A B C - переименовал заказы

CF2736-A - в названия нолики добавил
CF2736-B
CF2736-C

в ненайденных путях смотрел где есть / и ручками раскидывал файлы короче

поковырял папки, в которых больше 4 фоток

еще бывает папка photos внутри
 

In [1]:
import pandas as pd
import matplotlib as plt

In [196]:
orders = pd.read_excel('MIKIZ Keys ordered & references (2022.06.07).xlsx', header = 1)
orders.rename(columns={"Order number # \nCustomer (SecurKeys…)": "sc_num", "Key code / Bitting" : "bitting", "Key type" : "type", "Key done \n(Oui / Non / En cours)" : "status"}, inplace=True)
orders.bitting = orders.bitting.str.lower()
orders.type = orders.type.str.upper()
orders.status = orders.status.str.upper()
orders.drop_duplicates(subset='sc_num', keep = False, inplace=True)

In [197]:
cols = ['sc_num', 'type', 'bitting']
orders = orders[cols]
orders = orders.replace(['X', 'x'], None).dropna()


In [198]:
from glob import glob
import os

sc_orders_dir = 'E:\mikiz\segmentation_dataset\photos_securkeys'
sc_orders = glob(sc_orders_dir + '\*', recursive = True)

sc_orders_table = orders.sc_num.to_list()

sc_orders_pure = [os.path.basename(order) for order in sc_orders]


In [199]:
import re

orders = orders.loc[orders.sc_num.str.contains('CF')]
orders = orders.loc[~orders.type.str.contains('\n')]
orders = orders.loc[~orders.sc_num.str.contains('\n')]
orders = orders.loc[~orders.bitting.str.contains('-')]
orders = orders.loc[~orders.type.str.contains('/')]
orders['path'] = None
for _, row in orders.iterrows():
    #proc order name
    if '-' in row.sc_num:
        parts = re.split(" *- *", row.sc_num)
        row.path = f'{parts[0]}\{parts[1]}'
    elif '_' in row.sc_num:
        parts = re.split(" *_ *", row.sc_num)
        row.path = f'{parts[0]}\{parts[1]}'
    else:
        row.path = row.sc_num
    
    #proc type name
    row.type = row.type.replace(" ", "")
    if '(' in row.type:
        parts = re.split("\(", row.type)
        row.type = parts[0]

    #proc bitting
    row.bitting = row.bitting.replace(" ", "")
    if '(' in row.bitting:
        parts = re.split("\(", row.bitting)
        row.bitting = parts[0]

    if '\n' in row.bitting:        
        if 'side' in row.bitting:
            digits = re.findall(r'\d+', row.bitting)
            if len(digits) > 1:
                row.bitting = '-'.join(digits)
            else:
                row.bitting = digits[0]

orders = orders.loc[~orders.bitting.str.contains('\n')]

In [ ]:
есть табличка с путями, типами и биттингами, теперь надо сопоставить файлы и если нашлись - перенести куда надо

In [166]:
orders.head()

,sc_num,type,bitting,path
248,CF000101,CA8,00202,CF000101
259,CF000120,LF23R,1313b,CF000120
265,CF000123,LF23R,aa133,CF000123
269,CF000133,AB98,bbieg7,CF000133
296,CF000164,LF23R,1abba,CF000164


In [179]:
paths_check = []
for _, row in orders.iterrows():
    path_to_test = f'{sc_orders_dir}/{row.path}'
    if not os.path.isdir(path_to_test):
        paths_check.append(row.path)

In [182]:
for path in paths_check:
    if '/' in path:
        print(path, end=" ")

CF000201/bis CF002217/bis CF002810/A CF002810/B CF002810/D CF002818/A CF002818/B CF002818/C CF002831/A CF002831/B CF002844/A CF002844/B CF002845/A CF002845/B CF002845/C CF002845/D CF002878/A CF002907/B CF002907/E CF002940/A CF002940/B CF002940/C CF002965/A CF002965/B CF002971/A CF002971/B CF002974/B CF002980/A CF002988/A CF002988/B CF003003/A CF003003/B CF003004/B CF003050/A CF003050/B CF003192/A CF003192/B CF003192/C CF003206/B CF003226/A CF003297/A CF003316/A CF003330/B CF003415/A CF003453/A CF003453/B CF003517/A CF003578/B CF003702/A CF003702/B CF003702/C CF003702/D CF003702/E CF003768/B CF003841/A CF003924/A CF003924/B CF003924/C CF003924/D CF003932/A CF003932/B CF004012/A CF004012/B CF004023/A CF004089/A CF004087/A CF004087/B CF004095/A CF004119/A CF004134/A CF004137/B CF004153/A CF004153/B CF004206/B CF004216/B CF004217/A CF004217/B CF004217/C CF004116/A CF004116/B CF004116/C CF004233/A CF004251/B CF004312/A CF004312/B CF004312/C CF004335/A CF004476/A CF004478/B CF004478/C CF0044

In [ ]:
пробежимся по папкам, если в какой-нибудь из внетрунних папок файлов больше 4 - значит не разделили

In [200]:
sc_orders_dir = 'E:\mikiz\segmentation_dataset\photos_securkeys'
sc_orders = glob(sc_orders_dir + '\*', recursive = True)

In [187]:
for sub_dir in sc_orders:
    n_in_order = glob(sub_dir + '\*', recursive = True)
    subfolders_here = False
    for e in n_in_order:
        if not os.path.isfile(e):
            subfolders_here = True
            n_in_suborder = glob(e + '\*', recursive = True)
            if len(n_in_suborder) > 4:
                print(e)
    if not subfolders_here:
        if len(n_in_order) > 4:
            print(sub_dir)

E:\mikiz\segmentation_dataset\photos_securkeys\CF002004
E:\mikiz\segmentation_dataset\photos_securkeys\CF002644
E:\mikiz\segmentation_dataset\photos_securkeys\CF002728
E:\mikiz\segmentation_dataset\photos_securkeys\CF003022
E:\mikiz\segmentation_dataset\photos_securkeys\CF003101
E:\mikiz\segmentation_dataset\photos_securkeys\CF003416
E:\mikiz\segmentation_dataset\photos_securkeys\CF003702
E:\mikiz\segmentation_dataset\photos_securkeys\CF003918
E:\mikiz\segmentation_dataset\photos_securkeys\CF003928
E:\mikiz\segmentation_dataset\photos_securkeys\CF003986\A
E:\mikiz\segmentation_dataset\photos_securkeys\CF004087
E:\mikiz\segmentation_dataset\photos_securkeys\CF004150
E:\mikiz\segmentation_dataset\photos_securkeys\CF004233\B
E:\mikiz\segmentation_dataset\photos_securkeys\CF004346
E:\mikiz\segmentation_dataset\photos_securkeys\CF004426
E:\mikiz\segmentation_dataset\photos_securkeys\CF004613
E:\mikiz\segmentation_dataset\photos_securkeys\CF004799
E:\mikiz\segmentation_dataset\photos_securke

прошелся по этим папкам, разнес фотки/поудалял лишнее

теперь строим валидированный датасет

In [209]:
import shutil

sc_orders_validated = 'E:\mikiz\segmentation_dataset\photos_securkeys_validated'
paths_check = []
for _, row in orders.iterrows():
    path_to_test = f'{sc_orders_dir}\{row.path}'
    if os.path.isdir(path_to_test):
        dst_dir = f'{sc_orders_validated}\{row.type}\{row.bitting}'
        os.makedirs(dst_dir, exist_ok=True)
        files_order = glob(path_to_test + '\*', recursive = True)
        for f in files_order:
            shutil.copy(f, dst_dir)        

отобрать 100 фоток рандомных для тренировки детектора


In [3]:
from glob import glob
sc_orders_validated = 'E:\mikiz\segmentation_dataset\photos_securkeys_validated'
all_photos = glob(sc_orders_validated + '\*\*\*', recursive = True)

In [ ]:
import numpy as np
photos_indecies = np.random.randint(1, len(all_photos) + 1, 300)
rand_photos = [all_photos[i] for i in photos_indecies]

In [13]:
import shutil

sc_orders_validated = 'E:\mikiz\segmentation_dataset\object_detection'

for path in rand_photos:
    shutil.copy(path, sc_orders_validated)

In [6]:
# уже добавленные
sc_orders_added = 'E:\mikiz\segmentation_dataset\object_detection'
photos_added = glob(sc_orders_added + '\*', recursive = True)

In [11]:
import os
photo_names = [os.path.basename(photo) for photo in photos_added]

photos_without_added = []
for photo in all_photos:
    photo_name = os.path.basename(photo)
    if not photo_name in photo_names:
        photos_without_added.append(photo)

In [15]:
len(photos_added) + len(photos_without_added) - len(all_photos)

0

In [16]:
import numpy as np
np.random.seed(2)  
test_indecies = np.random.choice(len(photos_without_added), size=100, replace=False)
test_photos = [photos_without_added[i] for i in test_indecies]

In [19]:
import shutil

sc_orders_validated = "E:\mikiz\segmentation_dataset\object_detection\\test"

for path in test_photos:
    shutil.copy(path, sc_orders_validated)

Берем весь валидированный сет, прогоняем через модельку, обрезаем и сохраняем 
(сохраняем все обрезанное в одну папку + список уже сделанных файлов)

In [20]:
os.path.basename(all_photos[0])

'CF003416_1_e61cc78c309dc0b51d2bbabd14d93690.jpg'

In [104]:
#reload module magic without kernel restating
import importlib
import proc
importlib.reload(proc)
from proc import key_detection

In [105]:
from proc import key_detection
detectron = key_detection()
im = detectron.proc_img(all_photos[0])

load checkpoint from local path: E:\develop\mikiz\repos\dataset_prep\model_detection.pth
[array([[9.5323224e+02, 1.3073678e+03, 1.3351545e+03, 2.0885203e+03,
        9.9878329e-01],
       [4.3255512e+01, 5.5764828e+01, 5.3715576e+01, 7.5169243e+01,
        2.9410886e-02]], dtype=float32)]
[array([[9.5323224e+02, 1.3073678e+03, 1.3351545e+03, 2.0885203e+03,
        9.9878329e-01]], dtype=float32)]


In [65]:
len(done)

96

In [90]:
im = detectron.proc_img(all_photos[177])

[array([[1.0373236e+03, 1.1696287e+03, 1.9551764e+03, 2.9157397e+03,
        9.9862635e-01]], dtype=float32)]
[array([[1.0373236e+03, 1.1696287e+03, 1.9551764e+03, 2.9157397e+03,
        9.9862635e-01]], dtype=float32)]


In [107]:
len(not_done) + len(done)

3180

In [110]:
len(not_done)

125

In [27]:
done = []
not_done = []

0

In [81]:
t[0]

array([1.0354203e+03, 1.1858397e+03, 2.0037148e+03, 2.7401309e+03,
       8.7631714e-01], dtype=float32)

In [83]:
t = [np.array([1.0354203e+03, 1.1858397e+03, 2.0037148e+03, 2.7401309e+03,
       8.7631714e-01], dtype=np.float32), np.array([  0.63525885,   0.        ,  29.847271  , 120.70471   ,
         0.2055483 ], dtype=np.float32)]

print(sorted(t, key = lambda x: x[4], reverse=True)[0])

[1.0354203e+03 1.1858397e+03 2.0037148e+03 2.7401309e+03 8.7631714e-01]


In [109]:
folder_path = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped'
import cv2
not_done = []

for photo in all_photos:
    #if photo not in done and photo not in not_done:
    if photo not in done:
        print(photo)

        im = detectron.proc_img(photo)
        if im is not None:
            if cv2.imwrite(f'{folder_path}\\cropped_{os.path.basename(photo)}', im):
                done.append(photo)
            else:
                print(f'error - {photo}')
        else:
            not_done.append(photo)

E:\mikiz\segmentation_dataset\photos_securkeys_validated\AB1\bifce\CF005708_2_adf4d91707c400b1a45ff367076ef5a9.jpg
[[[]]]
[array([], shape=(1, 0), dtype=float64)]
E:\mikiz\segmentation_dataset\photos_securkeys_validated\AB20\3521\CF004433_3_4996d1c6392cc4e0098b75427a0c5b3a.jpg
[[[]]]
[array([], shape=(1, 0), dtype=float64)]
E:\mikiz\segmentation_dataset\photos_securkeys_validated\BD9R\21431\CF005985_2_33249e0316a76605df12cc6963383b4d.jpg
[array([[1.0363180e+03, 1.1652224e+03, 1.9679833e+03, 2.8363459e+03,
        9.9999523e-01],
       [0.0000000e+00, 2.2125423e-02, 2.9727936e+01, 1.1436663e+02,
        3.1197023e-02]], dtype=float32)]
[array([[1.0363180e+03, 1.1652224e+03, 1.9679833e+03, 2.8363459e+03,
        9.9999523e-01]], dtype=float32)]
E:\mikiz\segmentation_dataset\photos_securkeys_validated\BD9R\21431\CF005985_3_33249e0316a76605df12cc6963383b4d.jpg
[array([[9.9745026e+02, 1.1176305e+03, 2.1446775e+03, 3.0170999e+03,
        9.9267781e-01]], dtype=float32)]
[array([[9.9745026e+

In [ ]:
        im = detectron.proc_img(photo)
        if im is not None:
            if cv2.imwrite(f'{folder_path}\\cropped_{os.path.basename(photo)}', im):
                done.append(photo)
            else:
                print(f'error - {photo}')
        else:
            not_done.append(photo)

ресайзим фоточки, длинная сторона до 1200

In [111]:
from glob import glob
path_cropped = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped'
photos_cropped = glob(path_cropped + '\*', recursive = True)

In [114]:
folder_path = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped_resized'
import cv2

for photo in photos_cropped:
    dim = 1024
    img = mmcv.imread(photo)
    y = img.shape[0]
    x = img.shape[1]
    if x > y:
        img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        ratio = y/x
    else:
        ratio = x/y
    
    img = mmcv.imresize(img, (int(ratio*dim), dim))

    cv2.imwrite(f'{folder_path}\\resized_cropped_{os.path.basename(photo)}', img)
        

split to train/test/val (60 30 10)

In [16]:
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split

path_resized = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped_resized'
photos_resized = glob(path_resized + '\*.*', recursive = True)

X_train, X_test = train_test_split(photos_resized, test_size=0.3, random_state=1)

X_train, X_val = train_test_split(X_train, test_size=0.15, random_state=1) #0.7*0.15 ~ 0.1

In [17]:
import shutil

for part in zip([X_train, X_test, X_val], ['train', 'test', 'val']):
    for f_name in part[0]:
        #print(f'{path_resized}\\{part[1]}\\{os.path.basename(f_name)}')
        shutil.copy(f_name, f'{path_resized}\\{part[1]}\\{os.path.basename(f_name)}')